In [55]:
pip install pyarrow

In [57]:
import pandas as pd
import os

In [59]:
# Ruta base del proyecto 
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))

In [61]:
# Rutas de entrada (BRONZE)
ruta_bronze = os.path.join(ruta_base, "BRONZE")
csv_aceite = os.path.join(ruta_bronze, "aceite_condensado.csv")
csv_gas = os.path.join(ruta_bronze, "gas_natural.csv")


In [65]:
# Rutas de salida (SILVER)
ruta_silver = os.path.join(ruta_base, "SILVER")
os.makedirs(ruta_silver, exist_ok=True)

In [67]:
# Cargar los CSV
df_aceite = pd.read_csv(csv_aceite)
df_gas = pd.read_csv(csv_gas)

In [69]:
df_aceite.head(2)

,fecha,cuenca,ubicacion,campo,operador,liquidos_mbd,petroleo_mbd,condensado_mbd
0,1960-01-01,Burgos,Terrestre,BRASIL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.000000,0.000000,NaN
1,1960-01-01,Burgos,Terrestre,FRANCISCO CANO,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.418995,0.418995,NaN


In [71]:
print("Min y max año:", pd.to_datetime(df_aceite["fecha"]).dt.year.min(), "→", pd.to_datetime(df_aceite["fecha"]).dt.year.max())

Min y max año: 1960 → 2025


In [73]:
# Quitar nulos
df_aceite = df_aceite.dropna()
df_gas = df_gas.dropna()

In [75]:
# =======================
# 1. Revisión inicial
# =======================
print("Revisión inicial:")

print(f"Campos únicos en aceite: {df_aceite['campo'].nunique()}")
print(f"Operadores únicos en aceite: {df_aceite['operador'].nunique()}")

print(f"Campos únicos en gas: {df_gas['campo'].nunique()}")
print(f"Operadores únicos en gas: {df_gas['operador'].nunique()}")

Revisión inicial:
Campos únicos en aceite: 494
Operadores únicos en aceite: 36
Campos únicos en gas: 493
Operadores únicos en gas: 36


In [77]:
# 1. Obtener sets únicos de campos
campos_aceite = set(df_aceite["campo"].unique())
campos_gas = set(df_gas["campo"].unique())

# 2. Campos que están solo en aceite
solo_aceite = campos_aceite - campos_gas
print(f"Campos que producen solo aceite ({len(solo_aceite)}):")
print(sorted(solo_aceite))

# 3. Campos que están solo en gas
solo_gas = campos_gas - campos_aceite
print(f"\nCampos que producen solo gas ({len(solo_gas)}):")
print(sorted(solo_gas))

# 4. Campos que producen ambos
ambos = campos_aceite & campos_gas
print(f"\nCampos que producen tanto aceite como gas ({len(ambos)}):")
print(sorted(ambos)[:10], "...")  # Mostramos solo los primeros 10 para no saturar


Campos que producen solo aceite (1):
['XAXAMANI']

Campos que producen solo gas (0):
[]

Campos que producen tanto aceite como gas (493):
['ABKATUN', 'ACAHUAL', 'ACATIL', 'ACTUL', 'ACUATEMPA', 'AGAVE', 'AGUA FRIA', 'AGUA NACIDA', 'AGUACATE', 'AHUATEPEC'] ...


In [79]:
# Filtrar registros del campo XAXAMANI
xaxamani = df_aceite[df_aceite["campo"] == "XAXAMANI"]

# Calcular producción total y promedio
total_xaxamani = xaxamani["petroleo_mbd"].sum().round(2)
promedio_xaxamani = xaxamani["petroleo_mbd"].mean().round(2)
conteo_dias = xaxamani.shape[0]

print(f"Producción total de XAXAMANI: {total_xaxamani} millones de barriles diarios (Mbd)")
print(f"Promedio diario: {promedio_xaxamani} Mbd (basado en {conteo_dias} días)")


Producción total de XAXAMANI: 0.02 millones de barriles diarios (Mbd)
Promedio diario: 0.02 Mbd (basado en 1 días)


In [81]:
# Filtrar registros del campo XAXAMANI
xaxamani = df_aceite[df_aceite["campo"] == "XAXAMANI"].copy()

# Asegurar formato de fecha
xaxamani["fecha"] = pd.to_datetime(xaxamani["fecha"])

# Periodo de producción
fecha_inicio = xaxamani["fecha"].min()
fecha_fin = xaxamani["fecha"].max()

# Total y promedio
total = xaxamani["petroleo_mbd"].sum().round(2)
promedio = xaxamani["petroleo_mbd"].mean().round(2)
conteo_dias = xaxamani.shape[0]

# Operador y cuenca (tomamos el primero porque debe ser constante)
operador = xaxamani["operador"].iloc[0]
cuenca = xaxamani["cuenca"].iloc[0]

# Mostrar resumen
print("Resumen del campo XAXAMANI:")
print(f"Periodo de producción: {fecha_inicio.strftime('%B %Y')} → {fecha_fin.strftime('%B %Y')}")
print(f"Operador: {operador}")
print(f"Cuenca: {cuenca}")
print(f"Producción total: {total} millones de barriles diarios (Mbd)")
print(f"Promedio diario: {promedio} Mbd (en {conteo_dias} días de registro)")


Resumen del campo XAXAMANI:
Periodo de producción: August 2019 → August 2019
Operador: HOKCHI ENERGY
Cuenca: Cuencas del Sureste
Producción total: 0.02 millones de barriles diarios (Mbd)
Promedio diario: 0.02 Mbd (en 1 días de registro)


In [83]:
# Contar registros con producción = 0
ceros_aceite = (df_aceite["petroleo_mbd"] == 0).sum()
ceros_gas = (df_gas["gas_natural_sin_nitrogeno_mmpcd"] == 0).sum()

In [85]:
print(f" Registros con producción de petróleo = 0: {ceros_aceite}")
print(f" Registros con producción de gas = 0: {ceros_gas}")


 Registros con producción de petróleo = 0: 17641
 Registros con producción de gas = 0: 87


In [87]:

# Filtrar producción mayor a 0
df_aceite_limpio = df_aceite[df_aceite["petroleo_mbd"] > 0]
df_gas_limpio = df_gas[df_gas["gas_natural_sin_nitrogeno_mmpcd"] > 0]

In [89]:
# Contar registros con producción = 0
ceros_aceite = (df_aceite_limpio["petroleo_mbd"] == 0).sum()
ceros_gas = (df_gas_limpio["gas_natural_sin_nitrogeno_mmpcd"] == 0).sum()

In [91]:
print(f"Registros con producción de petróleo = 0: {ceros_aceite}")
print(f"Registros con producción de gas = 0: {ceros_gas}")

Registros con producción de petróleo = 0: 0
Registros con producción de gas = 0: 0


In [93]:
print("Min y max año:", pd.to_datetime(df_aceite_limpio["fecha"]).dt.year.min(), "→", pd.to_datetime(df_aceite_limpio["fecha"]).dt.year.max())

Min y max año: 2016 → 2025


In [95]:
df_aceite_limpio.head()

,fecha,cuenca,ubicacion,campo,operador,liquidos_mbd,petroleo_mbd,condensado_mbd
170266,2016-01-01,Cinturon Plegado de Chiapas,Terrestre,CERRO NANCHITAL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.582716,0.582716,0.0
170267,2016-01-01,Cuencas del Sureste,Aguas someras,MAY,PEMEX EXPLORACIÓN Y PRODUCCIÓN,15.446212,15.446212,0.0
170268,2016-01-01,Cuencas del Sureste,Aguas someras,ABKATUN,PEMEX EXPLORACIÓN Y PRODUCCIÓN,9.127253,9.127253,0.0
170269,2016-01-01,Cuencas del Sureste,Aguas someras,AKAL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,80.170865,80.170865,0.0
170270,2016-01-01,Cuencas del Sureste,Aguas someras,AYATSIL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,9.870175,9.870175,0.0


In [99]:
# Limpiar campos de texto en df_aceite_limpio correctamente
df_aceite_limpio.loc[:, "operador"] = df_aceite_limpio["operador"].str.strip().str.upper()
df_aceite_limpio.loc[:, "campo"] = df_aceite_limpio["campo"].str.strip().str.upper()
df_aceite_limpio.loc[:, "cuenca"] = df_aceite_limpio["cuenca"].str.strip().str.upper()
df_aceite_limpio.loc[:, "ubicacion"] = df_aceite_limpio["ubicacion"].str.strip().str.upper()


In [105]:

# Limpiar campos de texto en df_gas_limpio
df_gas_limpio.loc[:, "operador"] = df_gas_limpio["operador"].str.strip().str.upper()
df_gas_limpio.loc[:, "campo"] = df_gas_limpio["campo"].str.strip().str.upper()
df_gas_limpio.loc[:, "cuenca"] = df_gas_limpio["cuenca"].str.strip().str.upper()
df_gas_limpio.loc[:, "ubicacion"] = df_gas_limpio["ubicacion"].str.strip().str.upper()

In [107]:
# Guardar en formato Parquet
parquet_aceite = os.path.join(ruta_silver, "aceite_condensado.parquet")
parquet_gas = os.path.join(ruta_silver, "gas_natural.parquet")

In [109]:
df_aceite_limpio.to_parquet(parquet_aceite, index=False)
df_gas_limpio.to_parquet(parquet_gas, index=False)

In [ ]:
print("✅ Datos limpios guardados en formato Parquet en la carpeta SILVER.")
